# Assignment: 1 - Camera Calibration 

### Importing necessary libraries

In [6]:
import numpy as np
import pandas as pd
import scipy

### Normalization

In [7]:
def Normalization(nd, x):
    '''
    Normalization of coordinates (centroid to the origin and mean distance of sqrt(2 or 3).

    Input
    -----
    nd: number of dimensions, 3 here
    x: the data to be normalized (directions at different columns and points at rows)
    Output
    ------
    Tr: the transformation matrix (translation plus scaling)
    x: the transformed data
    '''

    x = np.asarray(x)
    m, s = np.mean(x, 0), np.std(x)
    if nd == 2:
        Tr = np.array([[s, 0, m[0]], [0, s, m[1]], [0, 0, 1]])
    else:
        Tr = np.array([[s, 0, 0, m[0]], [0, s, 0, m[1]], [0, 0, s, m[2]], [0, 0, 0, 1]])
        
    Tr = np.linalg.inv(Tr)
    x = np.dot( Tr, np.concatenate( (x.T, np.ones((1,x.shape[0]))) ) )
    x = x[0:nd, :].T

    return Tr, x

### DLT Camera Calibration

In [29]:
def DLTcalib(nd, xyz, uv):
    '''
    Camera calibration by DLT using known object points and their image points.

    Input
    -----
    nd: dimensions of the object space, 3 here.
    xyz: coordinates in the object 3D space.
    uv: coordinates in the image 2D space.

    The coordinates (x,y,z and u,v) are given as columns and the different points as rows.

    There must be at least 6 calibration points for the 3D DLT.

    Output
    ------
     L: array of 11 parameters of the calibration matrix.
     err: error of the DLT (mean residual of the DLT transformation in units of camera coordinates).
    '''
    if (nd != 3):
        raise ValueError('%dD DLT unsupported.' %(nd))
    
    # Converting all variables to numpy array
    xyz = np.asarray(xyz)
    uv = np.asarray(uv)

    n = xyz.shape[0]

    print("\n\nNUMBER OF POINTS: {}\n\n".format(n))
    # Validating the parameters:
    if uv.shape[0] != n:
        raise ValueError('Object (%d points) and image (%d points) have different number of points.' %(n, uv.shape[0]))

    if (xyz.shape[1] != 3):
        raise ValueError('Incorrect number of coordinates (%d) for %dD DLT (it should be %d).' %(xyz.shape[1],nd,nd))

    if (n < 6):
        raise ValueError('%dD DLT requires at least %d calibration points. Only %d points were entered.' %(nd, 2*nd, n))
        
    # Normalize the data to improve the DLT quality (DLT is dependent of the system of coordinates).
    # This is relevant when there is a considerable perspective distortion.
    # Normalization: mean position at origin and mean distance equals to 1 at each direction.
    Txyz, xyzn = Normalization(nd, xyz)
    Tuv, uvn = Normalization(2, uv)

    A = []

    for i in range(n):
        x, y, z = xyzn[i, 0], xyzn[i, 1], xyzn[i, 2]
        u, v = uvn[i, 0], uvn[i, 1]
        A.append( [x, y, z, 1, 0, 0, 0, 0, -u * x, -u * y, -u * z, -u] )
        A.append( [0, 0, 0, 0, x, y, z, 1, -v * x, -v * y, -v * z, -v] )

    # Convert A to array
    A = np.asarray(A) 

    # Find the 11 parameters:
    U, S, V = np.linalg.svd(A)

    # The parameters are in the last line of Vh and normalize them
    L = V[-1, :] / V[-1, -1]
    #print(L)
    # Camera projection matrix
    H = L.reshape(3, nd + 1)
    #print(H)

    # Denormalization
    # pinv: Moore-Penrose pseudo-inverse of a matrix, generalized inverse of a matrix using its SVD
    H = np.dot( np.dot( np.linalg.pinv(Tuv), H ), Txyz )
    #print(H)
    H = H / H[-1, -1]
    #print(H)
    L = H.flatten()
    #print(L)

    # Mean error of the DLT (mean residual of the DLT transformation in units of camera coordinates):
    uv2 = np.dot( H, np.concatenate( (xyz.T, np.ones((1, xyz.shape[0]))) ) ) 
    uv2 = uv2 / uv2[2, :] 
    # Mean distance:
    projected_pts = uv2.T
    actual_pixels = uv

    print('\n\nACTUAL PTS   |             ESTIMATED PTS')
    for i in range(len(uv2.T)):
        print("{}   <---->  {}".format(actual_pixels[i], projected_pts[i]))

    err = np.sqrt( np.mean(np.sum( (uv2[0:2, :].T - uv)**2, 1)) ) 

    return L, err

In [36]:
def get_intrest_points():
    # x = [20,  0, 0, 20, 20, 0, 0 , 0, 12.5, 5]
    # y = [0,   20, 0, 20, 0, 20, 0, 7.5,17.5, 0]
    # z = [0,   0, 20, 0, 20, 20, 0, 15, 0, 5]

    # u = [450, 137, 288, 294, 462, 115, 297, 237, 252, 328]
    # v = [630, 634, 382, 759, 417, 422, 552, 445, 690, 526]
    # WORLD CORDINATES
    data = pd.read_excel("dataset.xlsx")
    x = np.array(data['X'])
    y = np.array(data['Y'])
    z = np.array(data['Z'])

    u = np.array(data['x'])
    v= np.array(data['y'])

    world_pts = []
    pixel_pts = []

    # world_df = pd.DataFrame()
    # world_df["X"] = np.array(x)
    # world_df["Y"] = np.array(y)
    # world_df["Z"] = np.array(z)

    # pixel_df = pd.DataFrame()
    # pixel_df['x'] = np.array(u)
    # pixel_df['y'] = np.array(v)

    # world_df.to_excel("world_coordinates.xlsx")
    # pixel_df.to_excel("image_pixel_coordinates.xlsx")
    

    for i in range(10):
        world_pts.append([x[i], y[i], z[i]])
        pixel_pts.append([u[i], v[i]])

    print("DATA FRAME")
    print(data)

    # for i in range(len(world_pts)):
    #     print(world_pts[i],"  <---> ", pixel_pts[i])

    # from sklearn.utils import shuffle
    # world_pts, pixel_pts = shuffle(world_pts, pixel_pts, random_state = 0)


        
    print("\n\nWORLD points")
    print(world_pts)
    return world_pts, pixel_pts

def camera_param(P):
    K, R = np.linalg.qr(P)
    print("\n\nINTRINSIC MATRIX: ")
    print(K)
    print("\n\nROTATION MATRIX: ")
    print(R)
    

In [37]:

# Known 3D coordinates
xyz = [[-875, 0, 9.755], [442, 0, 9.755], [1921, 0, 9.755], [2951, 0.5, 9.755], [-4132, 0.5, 23.618],
[-876, 0, 23.618]]
# Known pixel coordinates
uv = [[76, 706], [702, 706], [1440, 706], [1867, 706], [264, 523], [625, 523]]

xyz, uv = get_intrest_points()
nd = 3
P, err = DLTcalib(nd, xyz, uv)

P = P.reshape(3,4)






DATA FRAME
      X     Y   Z    x    y
0  20.0   0.0   0  450  630
1   0.0  20.0   0  137  634
2   0.0   0.0  20  288  382
3  20.0  20.0   0  294  759
4  20.0   0.0  20  462  417
5   0.0  20.0  20  115  422
6   0.0   0.0   0  297  552
7   0.0   7.5  15  237  445
8  12.5  17.5   0  252  690
9   5.0   0.0   5  328  526


WORLD points
[[20.0, 0.0, 0], [0.0, 20.0, 0], [0.0, 0.0, 20], [20.0, 20.0, 0], [20.0, 0.0, 20], [0.0, 20.0, 20], [0.0, 0.0, 0], [0.0, 7.5, 15], [12.5, 17.5, 0], [5.0, 0.0, 5]]


NUMBER OF POINTS: 10




ACTUAL PTS   |             ESTIMATED PTS
[450 630]   <---->  [451.4072449  628.60318511   1.        ]
[137 634]   <---->  [138.46741682 633.95460016   1.        ]
[288 382]   <---->  [289.85100482 381.21856696   1.        ]
[294 759]   <---->  [294.81854532 761.04049839   1.        ]
[462 417]   <---->  [460.83635275 417.945411     1.        ]
[115 422]   <---->  [114.18965589 422.79992619   1.        ]
[297 552]   <---->  [295.58425721 553.17879701   1.        ]
[237 445

### Projection matrix (P)

In [38]:
print('Matrix')
print(P)

Matrix
[ 3.33886893e+00 -9.24702346e+00 -1.41415702e+00  2.95584257e+02
 -2.42876454e+00 -2.33054854e+00 -1.00809175e+01  5.53178797e+02
 -9.86311253e-03 -1.00469950e-02 -3.88991027e-03  1.00000000e+00]


### RMSE

In [39]:
print('\nError')
print(err)


Error
1.7846604247987097


### Camera Parameters

In [40]:
camera_param(P.reshape(3,4))



INTRINSIC MATRIX: 
[[-8.08676603e-01 -5.88253474e-01 -4.68022334e-05]
 [ 5.88248626e-01 -8.08669609e-01 -4.12526407e-03]
 [ 2.38885338e-03 -3.36353589e-03  9.99991490e-01]]


ROTATION MATRIX: 
[[-4.12880615e+00  6.10688555e+00 -4.78649947e+00  8.63769829e+01]
 [ 0.00000000e+00  7.32427124e+00  8.98402749e+00 -6.21220711e+02]
 [ 0.00000000e+00  0.00000000e+00  3.77627554e-02 -1.29585113e+00]]
